In [26]:
import numpy as np
import scipy.stats as stats
import statsmodels.api as sm
from gesla_functions import *
import os
import sys
import glob

This code takes high-frequency tide gauge records from GESLA 3, averages them as hourly records, select for a specific year and region, smooths them by means of a lowess filter to remove tide contributions and correct them for the Dynamic Atmospheric Correction. 

The whole processing is needed to make GESLA dataset comparable to altimetry estimations. The final output of this code can be used in compare_TG_nodac.

Note that the location and naming of the original GESLA data and of the DAC correction from AVISO are hard coded.

The following files will be created by the code (naming changes depending on the choices). Files 2 to 4 are only temporary files and removed at the end of the processing

1) A file containing a selection of the original GESLA data saved in xarray format in the location gesla_output_dir
This is a long process, if needed the parameter "decision_gesla" needs to be set as 'yes'

2) gesla_2021update_northsea.nc 
A selection of the original GESLA dataset in a specific region

3) gesla_2021_selected_update_northsea
Tide gauges with no data after the 15th of April of the selected year are excluded. If two tide gauges are closer than 1.5 km, only the longest record is kept.

4) gesla_2021_selected_lowessupdate_northsea
Lowess filter is applied to remove tides

5) gesla_2021_selected_lowess_dac_correctedupdate_northsea
DAC correction is applied



In [27]:

# Decide whether 
decision_gesla = 'no'


# Box in which data will be geographically selected 
max_lat = 90
min_lat = -90
max_lon = 180
min_lon = -180 

# Additional naming added in the output products
add = 'global'

# Location in which the selected gesla data will be saved 
gesla_output_dir = '/nfs/public_ads/Oelsmann/marcello/gesla_v3/'

# Location in which the DAC correction for the tide gauge data will be saved
out_dir='/nfs/public_ads/Oelsmann/marcello/gesla_v3/'

# Years to be considered in the tide gauge dataset
years_in = [2004]
start_date = pd.to_datetime('01-01-2004')

# Location where the corrected tide gauge xarray will be stored
tg_directory = '/DGFI8/H/work_marcello/machine_learning_altimetry_validation/'

In [28]:
def make_gesla(start_date = pd.to_datetime('01-01-2021'),add='update',gesla_output_dir = '/nfs/public_ads/Oelsmann/marcello/gesla_v3/',years_in = [2021]):
    dire = '/nfs/DGFI8/D/tide-gauges/GESLA_3/'
    #filenames = os.listdir(dire)
    meta = '/nfs/DGFI8/D/tide-gauges/GESLA_3_meta/GESLA3_ALL_2.csv'    
    gesla = GeslaDataset(meta,dire)
    #filenames = gesla.meta[gesla.meta['end_date_time'] > pd.to_datetime('31-12-2021')].filename.values
    filenames = gesla.meta.filename.values
    gesla_all = gesla.files_to_xarray(filenames = filenames,apply_use_flag=True,
                       start_date =start_date, resampling = 'H')
    gesla_all.to_netcdf(gesla_output_dir+'gesla_'+str(years_in[0])+add+'.nc')


def make_DAC_aviso(dataset,DAC_name='',name = 'dac_dif_26296_12.nc',dire = '/DGFI8/D/ib/DAC.AVISO/',years_in = [2021],out_dir='/nfs/public_ads/Oelsmann/marcello/gesla_v3/'):
    testfile = xr.open_dataset(dire +str(years_in[0])+'/'+ name)
    testfile=testfile.rename({'latitude':'lat','longitude':'lon'})
    lon_nw,lat_nw = np.meshgrid(testfile.lon,testfile.lat)
    start = np.full(lon_nw.shape,False)
    id1=[]
    id2=[]
    for lat,lon in zip(dataset.lat.values,dataset.lon.values):
        diff =abs((lon_nw-lon))+ abs((lat_nw-lat))
        out = np.where(diff == np.min(diff))
        id1.append(out[0][0])
        id2.append(out[1][0])
    for year in years_in:
        data  = glob.glob(dire+str(year)+'/*nc')
        DATA=np.empty((len(data),len(id1)))*np.nan
        TIME=pd.DataFrame(np.empty(len(data))*np.nan)
        i=0
        for dataname in data:
            print(i, dataname)
            dat = xr.open_dataset(dataname)

            DATA[i,:]=dat.dac.values[id1,id2]
            TIME.loc[i]=np.datetime64(dat.dac.date[:-4])     
            i=i+1
        DATA = pd.DataFrame(DATA)
        DATA['time']=TIME
        DATA.to_csv(out_dir+'data_for_time_'+str(year))

In [29]:
# Very long process!!!

if decision_gesla == 'yes' :

    make_gesla(start_date ,add, gesla_output_dir, years_in)

In [30]:
gesla_all = xr.open_dataset(gesla_output_dir+'gesla_'+str(years_in[0])+'australia_2004.nc')
gesla_all


<xarray.Dataset>
Dimensions:                  (date_time: 8783, station: 1757)
Coordinates:
  * station                  (station) int64 0 1 2 3 4 ... 1753 1754 1755 1756
  * date_time                (date_time) datetime64[ns] 2004-01-01T01:00:00 ....
Data variables: (12/27)
    sea_level                (station, date_time) float64 ...
    qc_flag                  (station, date_time) float64 ...
    use_flag                 (station, date_time) float64 ...
    filename                 (station) object ...
    site_name                (station) object ...
    site_code                (station) object ...
    ...                       ...
    datum_information        (station) object ...
    instrument               (station) object ...
    precision                (station) object ...
    null_value               (station) float64 ...
    gauge_type               (station) object ...
    overall_record_quality   (station) object ...

In [31]:
# Geographical limit

gesla_all = gesla_all.where((gesla_all.latitude >= min_lat) & (gesla_all.latitude <= max_lat) & (gesla_all.longitude >= min_lon) & (gesla_all.longitude <= max_lon), drop=True) 
# Loop through variables in the dataset and put unknown data types to string, otherwise "to_netcdf" does not work
for varname in gesla_all.variables:
    if gesla_all[varname].dtype not in ['float64', 'datetime64[ns]','int64']:
        gesla_all[varname] = gesla_all[varname].astype(dtype='U80', copy=False)

filename_and_path_limit = tg_directory+'gesla_'+str(years_in[0])+add+'.nc'
gesla_all.to_netcdf(filename_and_path_limit)
os.chmod(filename_and_path_limit, 0o777)

In [32]:
# Plot of a random gesla_all station

# gesla_all.info

# import xarray as xr
# import matplotlib.pyplot as plt
# import numpy as np

# %matplotlib inline

# # Assuming gesla_all is your xarray dataset
# # gesla_all = xr.open_dataset('path_to_your_dataset.nc')

# # Select a random station
# stations = gesla_all.station.values
# random_station = np.random.choice(stations)

# # Extract data for the selected random station
# sea_level_data = gesla_all.sel(station=random_station)

# # Plot date vs sea level for the selected station
# plt.figure(figsize=(10, 5))
# plt.plot(sea_level_data.date_time, sea_level_data.sea_level, label=f'Station {random_station}')
# plt.xlabel('Date')
# plt.ylabel('Sea Level (m)')
# plt.title(f'Sea Level at Station {random_station}')
# plt.legend()
# plt.grid(True)
# plt.show()


In [33]:
#Content of select_and_dropdupl
#Note that tide gauges with no data after the 15th of April of the selected year are excluded
#If two tide gauges are closer than 1.5 km, only the longest record is kept

gesla_all_sorted = gesla_all.sortby('site_name')
limit = 1500
counts = gesla_all_sorted['sea_level'].count(dim='date_time').values
coords =np.stack([gesla_all_sorted.latitude.values,gesla_all_sorted.longitude.values])
distancs = []
for coord1 in coords.T:
    distancs.append(haversine(coord1, coords))
distancs = np.vstack(distancs)
pairs = []
reject_indices = []
for i in range(distancs.shape[0]):
    sub = distancs[:,i]
    idx = np.argwhere(   (sub<limit) & (sub!=0) ) 
    if len(idx)>0:
        pairs.append([i,idx[0]])
        if counts[i]<=counts[int(idx[0])]:
            reject_indices.append(i)
        else:
            reject_indices.append(int(idx[0]))
reject_at  = pd.DataFrame(reject_indices).drop_duplicates().values.flatten()
gesla_selected = gesla_all_sorted.sel({'station':~np.isin(np.arange(distancs.shape[0]),reject_at)})
mask = (gesla_selected['date_time'].to_dataframe()['date_time'] >  pd.to_datetime('04-15-'+str(years_in[0]))).values
counts_after_april = gesla_selected.sel({'date_time':mask}).count(dim='date_time')['sea_level']
gesla_selected_final = gesla_selected.where(counts_after_april > 1,drop=True)
gesla_selected_final.attrs={'comment':'duplicates are rejected; longest TG record of a TG pair within 1.5km is kept. TG must contain data after 04-15-. Use-flag applied. Hourly averaged.'}

filename_and_path_dropupl = tg_directory+'gesla_'+str(years_in[0])+'_selected_'+add+'.nc'
gesla_selected_final.to_netcdf(filename_and_path_dropupl)
os.chmod(filename_and_path_dropupl, 0o777)

In [34]:
#apply_lowess_filter
import copy
lowess = sm.nonparametric.lowess
gesla_selected_final=xr.open_dataset(tg_directory+'gesla_'+str(years_in[0])+'_selected_'+add+'.nc')
gesla_selected_final['sea_level_lowess']=copy.deepcopy(gesla_selected_final['sea_level'])
frac = 40/len(gesla_selected_final.date_time)
x=np.arange(len(gesla_selected_final.date_time))
for i in range(len(gesla_selected_final.station)):
    y = gesla_selected_final['sea_level'][i,:].values
    w = lowess(y, x, frac=frac)
    gesla_selected_final['sea_level_lowess'][i,np.isin(x,w[:,0])] = w[:,1]
    print(i)
    
filename_and_path_lowess = tg_directory+'gesla_'+str(years_in[0])+'_selected_lowess'+add+'.nc'
gesla_selected_final.to_netcdf(filename_and_path_lowess)
os.chmod(filename_and_path_lowess, 0o777)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [35]:
gesla_selected_final

<xarray.Dataset>
Dimensions:                  (date_time: 8783, station: 1067)
Coordinates:
  * station                  (station) int64 478 818 872 870 ... 418 741 359 159
  * date_time                (date_time) datetime64[ns] 2004-01-01T01:00:00 ....
Data variables: (12/28)
    sea_level                (station, date_time) float64 ...
    qc_flag                  (station, date_time) float64 ...
    use_flag                 (station, date_time) float64 ...
    filename                 (station) object ...
    site_name                (station) object ...
    site_code                (station) object ...
    ...                       ...
    instrument               (station) object ...
    precision                (station) object ...
    null_value               (station) float64 ...
    gauge_type               (station) object ...
    overall_record_quality   (station) object ...
    sea_level_lowess         (station, date_time) float64 ...
Attributes:
    comment:  duplicates are rejected; longest TG record of a TG pair within ...

In [36]:
#compute_dac_gesla
gesla = xr.open_dataset(tg_directory+'gesla_'+str(years_in[0])+'_selected_lowess'+add+'.nc')
dataset= gesla
dataset = dataset.rename({'latitude':'lat','longitude':'lon'})
lon= dataset['lon']
lon[lon <0 ] = lon[lon <0 ]+360
dataset['lon'][:] = lon

# Define the directory path
directory_path = '/DGFI8/D/ib/DAC.AVISO/'+str(years_in[0])+'/'
print(directory_path)

# Define the pattern to match files that start with "dac_dif"
pattern = os.path.join(directory_path, 'dac_dif*.nc')

# List all files matching the pattern
files = glob.glob(pattern)

# Check if there are any files matching the pattern
if files:
    # Randomly select a file from the list
    name = np.random.choice(files)
    
    # Extract the filename from the full path
    name = os.path.basename(name)
    
    print(name)




make_DAC_aviso(dataset,DAC_name='',name = name ,dire = '/DGFI8/D/ib/DAC.AVISO/',years_in = years_in ,out_dir='/nfs/public_ads/Oelsmann/marcello/gesla_v3/')

0 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19757_00.nc
1 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19814_12.nc
2 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19898_18.nc
3 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19872_12.nc
4 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_20023_06.nc
5 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_20060_12.nc
6 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19938_06.nc
7 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19974_06.nc
8 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19811_00.nc
9 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_20041_06.nc
10 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19879_18.nc
11 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19925_06.nc
12 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19740_12.nc
13 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_20085_00.nc
14 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19847_18.nc
15 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19917_18.nc
16 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19819_06.nc
17 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19899_12.nc
18 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19824_12.nc
19 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19750_18.nc
20 /DGFI8/

166 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_20032_12.nc
167 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19909_06.nc
168 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19858_18.nc
169 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_20068_12.nc
170 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19747_12.nc
171 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19980_18.nc
172 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_20013_12.nc
173 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_20016_00.nc
174 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19764_00.nc
175 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19956_06.nc
176 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19779_06.nc
177 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19912_00.nc
178 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19757_12.nc
179 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19933_12.nc
180 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19790_00.nc
181 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19782_18.nc
182 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19937_00.nc
183 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19918_06.nc
184 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19850_00.nc
185 /DGFI8/D/ib/DAC.AVISO/2004/

328 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19846_00.nc
329 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19989_00.nc
330 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19913_00.nc
331 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19856_00.nc
332 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19887_12.nc
333 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19925_12.nc
334 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_20085_06.nc
335 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_20006_18.nc
336 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19875_06.nc
337 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19932_06.nc
338 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19828_12.nc
339 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_20062_00.nc
340 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_20046_12.nc
341 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19901_00.nc
342 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19994_18.nc
343 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19921_12.nc
344 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19948_18.nc
345 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19881_00.nc
346 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19735_06.nc
347 /DGFI8/D/ib/DAC.AVISO/2004/

492 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19961_06.nc
493 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_20066_12.nc
494 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_20017_06.nc
495 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19874_06.nc
496 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19809_18.nc
497 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19727_12.nc
498 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19723_12.nc
499 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19864_06.nc
500 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_20043_18.nc
501 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19806_12.nc
502 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19914_06.nc
503 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19938_12.nc
504 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_20048_12.nc
505 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_20066_18.nc
506 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19937_12.nc
507 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19772_00.nc
508 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19827_12.nc
509 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19838_18.nc
510 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_20002_12.nc
511 /DGFI8/D/ib/DAC.AVISO/2004/

653 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_20042_00.nc
654 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19922_12.nc
655 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_20011_12.nc
656 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19921_18.nc
657 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19931_18.nc
658 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19990_00.nc
659 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19850_18.nc
660 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19816_00.nc
661 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19869_12.nc
662 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19826_18.nc
663 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19949_12.nc
664 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19847_12.nc
665 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_20080_06.nc
666 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19770_12.nc
667 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19829_06.nc
668 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19913_18.nc
669 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_20050_12.nc
670 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_20025_06.nc
671 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19795_12.nc
672 /DGFI8/D/ib/DAC.AVISO/2004/

815 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19895_06.nc
816 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19796_06.nc
817 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_20007_06.nc
818 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_20002_06.nc
819 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19945_12.nc
820 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19934_18.nc
821 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_20075_18.nc
822 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_20029_18.nc
823 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19845_00.nc
824 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19982_18.nc
825 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_20043_00.nc
826 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_20032_18.nc
827 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19747_00.nc
828 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19972_00.nc
829 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19934_00.nc
830 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19846_12.nc
831 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_20015_00.nc
832 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_20055_00.nc
833 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19821_18.nc
834 /DGFI8/D/ib/DAC.AVISO/2004/

977 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_20020_06.nc
978 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19770_06.nc
979 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19841_18.nc
980 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19920_06.nc
981 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_20015_06.nc
982 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19971_18.nc
983 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_20079_00.nc
984 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19930_12.nc
985 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_20012_00.nc
986 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19961_00.nc
987 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19737_00.nc
988 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19755_18.nc
989 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_20053_12.nc
990 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_20037_18.nc
991 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19880_00.nc
992 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_20064_06.nc
993 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19894_12.nc
994 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19768_00.nc
995 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19837_00.nc
996 /DGFI8/D/ib/DAC.AVISO/2004/

1137 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19895_12.nc
1138 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19927_18.nc
1139 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19884_12.nc
1140 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19895_18.nc
1141 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19832_18.nc
1142 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19794_00.nc
1143 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_20079_06.nc
1144 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19932_18.nc
1145 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19937_18.nc
1146 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19889_06.nc
1147 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_20045_00.nc
1148 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_20065_18.nc
1149 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19842_00.nc
1150 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19797_00.nc
1151 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19934_12.nc
1152 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19825_12.nc
1153 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19938_00.nc
1154 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19797_06.nc
1155 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19948_00.nc
1156 /DGFI8/

1299 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19801_18.nc
1300 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19923_06.nc
1301 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_20068_00.nc
1302 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19756_06.nc
1303 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19813_06.nc
1304 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_20077_00.nc
1305 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19927_06.nc
1306 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19729_12.nc
1307 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19892_00.nc
1308 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19973_06.nc
1309 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19742_00.nc
1310 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19923_00.nc
1311 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19885_00.nc
1312 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19726_18.nc
1313 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_20035_18.nc
1314 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19872_06.nc
1315 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19875_12.nc
1316 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19813_00.nc
1317 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_20083_18.nc
1318 /DGFI8/

1458 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19919_00.nc
1459 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19725_06.nc
1460 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19853_06.nc
1461 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19941_12.nc
1462 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19990_06.nc
1463 /DGFI8/D/ib/DAC.AVISO/2004/dac_dif_19840_00.nc


In [37]:
# Here I first need to apply lowess!!!

#correct_gesla_for_dac()

dire = '/DGFI8/D/ib/DAC.AVISO/'


gesla = xr.open_dataset(tg_directory+'gesla_'+str(years_in[0])+'_selected_lowess'+add+'.nc')
dac = pd.read_csv(out_dir+'data_for_time_'+str(years_in[0])).sort_values(by='time')
dac=dac.set_index('time')
dac.index = pd.to_datetime(dac.index)
dac_sub = dac[(pd.DatetimeIndex(dac.index.values) >= pd.Timestamp(gesla['date_time'][0].values)) & (pd.DatetimeIndex(dac.index.values) <= pd.Timestamp(gesla['date_time'][-1].values))]
gesla_time = gesla.loc[dict(date_time=slice(dac.index[0], dac.index[-1]))]
gesla_time['dac'] = copy.deepcopy(gesla_time['sea_level_lowess'])*np.nan
dac_res = (dac_sub.resample('1H').mean()).interpolate(method='cubic',limit=10)
gesla_time['dac'][:,np.isin(gesla_time.date_time,dac_res.index)] = dac_res.iloc[:,1:].values.T
gesla_time['sla_dac']=gesla_time['sea_level_lowess']-gesla_time['dac']
gesla_time['sla_dac_no_loess']=gesla_time['sea_level']-gesla_time['dac']

filename_and_path = tg_directory+'gesla_'+str(years_in[0])+'_selected_lowess_dac_corrected'+add+'.nc'
gesla_time.to_netcdf(filename_and_path)
os.chmod(filename_and_path, 0o777)

os.remove(filename_and_path_limit)
os.remove(filename_and_path_dropupl)
os.remove(filename_and_path_lowess)

In [38]:
gesla_time

<xarray.Dataset>
Dimensions:                  (date_time: 8778, station: 1067)
Coordinates:
  * station                  (station) int64 478 818 872 870 ... 418 741 359 159
  * date_time                (date_time) datetime64[ns] 2004-01-01T01:00:00 ....
Data variables: (12/31)
    sea_level                (station, date_time) float64 ...
    qc_flag                  (station, date_time) float64 ...
    use_flag                 (station, date_time) float64 ...
    filename                 (station) object ...
    site_name                (station) object ...
    site_code                (station) object ...
    ...                       ...
    gauge_type               (station) object ...
    overall_record_quality   (station) object ...
    sea_level_lowess         (station, date_time) float64 ...
    dac                      (station, date_time) float64 nan nan ... -0.0109
    sla_dac                  (station, date_time) float64 nan nan ... 1.77 1.74
    sla_dac_no_loess         (station, date_time) float64 nan nan ... 2.906 2.52
Attributes:
    comment:  duplicates are rejected; longest TG record of a TG pair within ...